## Maro: ML Automated Remediation Oracle

Gives a basic example of how to use Maro and its features

In [2]:
import lale.datasets.openml
from sklearn.model_selection import train_test_split

from lale.lib.lale import Project
from lale.lib.lale import categorical
from lale.lib.sklearn import LogisticRegression
from lale.lib.sklearn import OneHotEncoder
from lale.lib.sklearn import StandardScaler
from lale.lib.lale import Hyperopt

from maro.maro import *

import warnings
warnings.filterwarnings('ignore')

In [3]:
# Dataset that we'll be using for the example
class CreditG:
    def __init__(self, train_size, missing_rate, stratify=False, random_state=None):
        (train_X, train_y), (test_X, test_y) = lale.datasets.openml.fetch(
            'credit-g', 'classification', preprocess=False)
        s = None
        if stratify:
            s = train_y
        subsample_X, _, subsample_y, _ = train_test_split(
            train_X, train_y, train_size=train_size, stratify=s, random_state=random_state)
        if missing_rate > 0.0:
            subsample_X = add_missing_values(subsample_X, missing_rate=missing_rate)
        self.train_X = subsample_X
        self.train_y = subsample_y
        self.test_X = test_X
        self.test_y = test_y
        
credit5 = CreditG(train_size=0.80, missing_rate=0.0)
train_X = credit5.train_X.to_numpy()
train_y = credit5.train_y
credit5.train_X.head()

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,residence_since,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker
697,no checking,12.0,critical/other existing credit,new car,926.0,<100,unemployed,1.0,female div/dep/mar,none,2.0,life insurance,38.0,none,own,1.0,unemp/unskilled non res,1.0,none,yes
353,<0,12.0,no credits/all paid,radio/tv,6199.0,<100,1<=X<4,4.0,male single,none,2.0,life insurance,28.0,none,rent,2.0,skilled,1.0,yes,yes
655,<0,14.0,existing paid,new car,3973.0,<100,unemployed,1.0,male single,none,4.0,no known property,22.0,none,for free,1.0,skilled,1.0,none,yes
235,<0,24.0,existing paid,radio/tv,1823.0,<100,unemployed,4.0,male single,none,2.0,car,30.0,stores,own,1.0,high qualif/self emp/mgmt,2.0,none,yes
218,<0,24.0,existing paid,furniture/equipment,3021.0,<100,1<=X<4,2.0,male div/sep,none,2.0,real estate,24.0,none,rent,1.0,unskilled resident,1.0,none,yes


In [4]:
# Pipeline that we'll be using for the example

one_hot_encoder = OneHotEncoder(handle_unknown='ignore')
planned = one_hot_encoder >> StandardScaler >> LogisticRegression

In [5]:
# Use AutoML to create initial evaluations to send to Maro

hyperopt_trainable = Hyperopt(estimator=planned, max_evals=20)
hyperopt_trained = hyperopt_trainable.fit(train_X, train_y)
hyperopt_trained.summary()

100%|█████████████████████| 20/20 [00:06<00:00,  3.22trial/s, best loss: -0.7238317757009346]
3 out of 20 trials failed, call summary() for details.
Run with verbose=True to see per-trial exceptions.


,tid,loss,time,log_loss,status
name,,,,,
p0,0,-0.712617,0.025218,0.730541,ok
p1,1,-0.714486,0.021421,0.729853,ok
p2,2,-0.721980,0.025734,0.571607,ok
p3,3,-0.714486,0.031548,0.730392,ok
p4,4,-0.721980,0.029015,0.571797,ok
p5,5,NaN,NaN,NaN,fail
p6,6,-0.721980,0.079145,0.573168,ok
p7,7,-0.714486,0.031369,0.730395,ok
p8,8,-0.723832,0.075429,0.613586,ok


In [8]:
remediated = auto_remediate(planned, hyperopt_trained, explanation=True)

Try setting argument 'with_mean' in operator StandardScaler to 'False'


In [9]:
planned.diff(remediated, ipython_display=True, customize_schema=True)

```diff
  from sklearn.preprocessing import OneHotEncoder
  from sklearn.preprocessing import StandardScaler
  from sklearn.linear_model import LogisticRegression
  import lale
  
  lale.wrap_imported_operators()
  one_hot_encoder = OneHotEncoder(handle_unknown="ignore")
+ standard_scaler = StandardScaler.customize_schema(with_mean={"enum": [False]})
- pipeline = one_hot_encoder >> StandardScaler >> LogisticRegression
?                               ^       ^

+ pipeline = one_hot_encoder >> standard_scaler >> LogisticRegression
?                               ^       ^^

```

In [10]:
# Re-run using remediated pipeline

hyperopt_trainable = Hyperopt(estimator=remediated, max_evals=20)
hyperopt_trained = hyperopt_trainable.fit(train_X, train_y)

100%|█████████████████████| 20/20 [00:05<00:00,  3.57trial/s, best loss: -0.7257009345794393]
